In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time

In [2]:
df = pd.read_csv('reviews_full_v5.csv')
df

,User,Date,Year,Month,Release,Band,Score,Link,Album_link,Review_title,Genre,Text,Sentiment
0,SpookyApparition,July 11 2002,2002,7,Chaosphere,Meshuggah,82,https://www.metal-archives.com/reviews/Meshugg...,https://www.metal-archives.com/albums/Meshugga...,Technical metal at it's finest.,technical thrash metal|math metal|progressive ...,I originally bought Chaosphere in late 2000......,0.9728
1,SpookyApparition,July 14 2002,2002,7,Necroticism_-_Descanting_the_Insalubrious,Carcass,88,https://www.metal-archives.com/reviews/Carcass...,https://www.metal-archives.com/albums/Carcass/...,Carcass at their finest.,grindcore|melodic death metal,A midpoint between Carcass' early grind albums...,0.8479
2,MarkRyan,July 15 2002,2002,7,Angels_Fall_First,Nightwish,90,https://www.metal-archives.com/reviews/Nightwi...,https://www.metal-archives.com/albums/Nightwis...,On par with their other stuff...,symphonic power metal,"I'm not hard-core into metal at all, really, b...",0.9951
3,Fear,July 16 2002,2002,7,Dark_Genesis,Iced_Earth,87,https://www.metal-archives.com/reviews/Iced_Ea...,https://www.metal-archives.com/albums/Iced_Ear...,Brilliance in Repackaging,heavy metal|us power metal,"This is not a new album, but in fact, five alb...",0.9976
4,Esoteric,July 16 2002,2002,7,A_Night_at_the_Opera,Blind_Guardian,70,https://www.metal-archives.com/reviews/Blind_G...,https://www.metal-archives.com/albums/Blind_Gu...,Somewhat Disappointing,power metal,I've been trying for months to come up with an...,0.5494
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93429,6CORPSE6GRINDER6,June 30 2017,2017,6,Chemical_Assault,Violator,85,https://www.metal-archives.com/reviews/Violato...,https://www.metal-archives.com/albums/Violator...,"Take your life back, in the pit!",thrash metal,I don't have any issues with the retro-thrash ...,0.9976
93430,6CORPSE6GRINDER6,June 30 2017,2017,6,Path_of_Fire,Aeon,65,https://www.metal-archives.com/reviews/Aeon/Pa...,https://www.metal-archives.com/albums/Aeon/Pat...,As mediocre as it's predecessor,death metal,Aeon’s brand of death metal is supposed to be ...,-0.9758
93431,6CORPSE6GRINDER6,June 30 2017,2017,6,Violent_Mosh,Violator,80,https://www.metal-archives.com/reviews/Violato...,https://www.metal-archives.com/albums/Violator...,Officers are dirtier than the criminals they c...,thrash metal,This band was so important for the thrash meta...,0.9911
93432,Wacke,June 30 2017,2017,6,Resurrection,Chimaira,89,https://www.metal-archives.com/reviews/Chimair...,https://www.metal-archives.com/albums/Chimaira...,What doesn't kill us makes us stronger.,hard rock,Chimaira is arguably the most overlooked and/o...,0.9947


In [4]:
genres_list = ['Black',
 'Death',
 'Doom/Stoner/Sludge',
 'Electronic/Industrial',
 'Experimental/Avant-garde',
 'Folk/Viking/Pagan',
 'Gothic',
 'Grindcore',
 'Groove',
 'Heavy',
 'Metalcore/Deathcore',
 'Power',
 'Progressive',
 'Speed',
 'Symphonic',
 'Thrash']
genres = set()
for genre in genres_list:
    for item in genre.split('/'):
        genres.add(item)
genres

{'Avant-garde',
 'Black',
 'Death',
 'Deathcore',
 'Doom',
 'Electronic',
 'Experimental',
 'Folk',
 'Gothic',
 'Grindcore',
 'Groove',
 'Heavy',
 'Industrial',
 'Metalcore',
 'Pagan',
 'Power',
 'Progressive',
 'Sludge',
 'Speed',
 'Stoner',
 'Symphonic',
 'Thrash',
 'Viking'}

In [5]:
bands = set()

In [9]:
mp = {}
for i,row in df.iterrows():
    mp[row['Band']] = row['Link']

In [13]:
len(bands)

20574

In [6]:
path = 'https://www.metal-archives.com/bands/'
genre = []
df1 = pd.read_csv('band_link.csv')
a = time.time()
for i,row in df1.iloc[:5].iterrows():#one third
    if i%1 == 0:
        print(i, time.time()-a)
    band = row['Band']
    url = path + str(band)
    r = requests.get(url)
    html_doc = r.content
    soup = BeautifulSoup(html_doc, 'html.parser')
    if soup.find(string = 'Formed in:') == None:
        urllink = row['Link']
        r = requests.get(urllink)
        html_doc = r.content
        soup_text = BeautifulSoup(html_doc, 'html.parser')
        url = [x['href'] for x in soup_text.find_all('a', href=True) if path in x['href']]
        if len(url) == 0:
            genre.append(np.NaN)
            continue
        else:
            url = url[0]
        r = requests.get(url)
        html_doc = r.content
        soup = BeautifulSoup(html_doc, 'html.parser')
        #print(url)
    item = soup.find(string = 'Genre:').find_next("dd").contents[0]
    ret = ""
    for x in genres:
        if x in item:
            ret += x +'|'
    genre.append(ret[:-1])
df1 = pd.DataFrame()
df1['Band_Genre'] = genre
df1.to_csv('genres_new0.csv', index = False)
time.time()-a

0 0.0029823780059814453
1 0.8926126956939697
2 2.640475034713745
3 3.4928805828094482
4 21.37393093109131


22.270654678344727

In [7]:
df1

,Band_Genre
0,Black
1,NaN
2,Sludge|Doom
3,Thrash|Speed
4,
